## 🔧 1. Setup do Ambiente

In [ ]:
# Imports necessários
import os
import sys
from pathlib import Path
from datetime import datetime

# Adiciona o diretório raiz ao path
PROJECT_ROOT = Path(os.getcwd()).parent
sys.path.insert(0, str(PROJECT_ROOT))

# Spark e Delta
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Visualização
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações de visualização
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print(f"📁 Project Root: {PROJECT_ROOT}")
print(f"📅 Data da Execução: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Inicialização do Spark Session
spark = (SparkSession.builder
    .appName("ABInBev_Demo_Presentation")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.driver.memory", "4g")
    .config("spark.sql.shuffle.partitions", "8")
    .getOrCreate())

print(f"✅ Spark Version: {spark.version}")
print(f"✅ Spark UI: {spark.sparkContext.uiWebUrl}")

In [ ]:
# Definição dos Paths
DATA_PATH = PROJECT_ROOT / "data"

# Camadas da Arquitetura Medallion
PATHS = {
    "landing": DATA_PATH / "landing",
    "bronze": DATA_PATH / "bronze",
    "silver": DATA_PATH / "silver",
    "gold": DATA_PATH / "gold",
    "consumption": DATA_PATH / "consumption",
    "control": DATA_PATH / "control"
}

print("📂 Estrutura de Diretórios:")
for layer, path in PATHS.items():
    exists = "✅" if path.exists() else "❌"
    print(f"  {exists} {layer.upper()}: {path}")

## 🏗️ 2. Visão Geral da Arquitetura

```
┌─────────────────────────────────────────────────────────────────────────────────┐
│                        ARQUITETURA MEDALLION - ABInBev                          │
├─────────────────────────────────────────────────────────────────────────────────┤
│                                                                                 │
│  ┌─────────┐    ┌─────────┐    ┌─────────┐    ┌─────────┐    ┌─────────────┐   │
│  │ LANDING │ -> │ BRONZE  │ -> │ SILVER  │ -> │  GOLD   │ -> │ CONSUMPTION │   │
│  │  (Raw)  │    │(Ingested)│   │(Cleaned)│    │(Business)│   │(Dimensional)│   │
│  └─────────┘    └─────────┘    └─────────┘    └─────────┘    └─────────────┘   │
│       │              │              │              │                │          │
│    CSV/TXT       Parquet        Parquet        Parquet          Parquet       │
│                  + Audit       + Quality      + Business       + Star Schema  │
│                                                                               │
├─────────────────────────────────────────────────────────────────────────────────┤
│  TABELAS:                                                                       │
│  • beverage_sales (4.5K+ registros)                                            │
│  • channel_features (características de canais)                                │
│  • Dimensões: dim_time, dim_product, dim_region, dim_channel                   │
│  • Fatos: fact_sales                                                           │
│  • Agregações: agg_sales_by_month, agg_sales_by_trade_group, etc.             │
└─────────────────────────────────────────────────────────────────────────────────┘
```

## 📊 3. Análise das Camadas de Dados

### 3.1 🥉 Camada BRONZE - Dados Brutos Ingeridos

In [ ]:
# Carregamento dos dados Bronze
bronze_sales = spark.read.parquet(str(PATHS["bronze"] / "beverage_sales"))
bronze_channels = spark.read.parquet(str(PATHS["bronze"] / "channel_features"))

print("📊 BRONZE LAYER - Estatísticas")
print("=" * 50)
print(f"\n🍺 Beverage Sales:")
print(f"   • Total de registros: {bronze_sales.count():,}")
print(f"   • Total de colunas: {len(bronze_sales.columns)}")
print(f"\n📺 Channel Features:")
print(f"   • Total de registros: {bronze_channels.count():,}")
print(f"   • Total de colunas: {len(bronze_channels.columns)}")

In [ ]:
# Schema da tabela Bronze Sales
print("📋 Schema - Bronze Beverage Sales:")
bronze_sales.printSchema()

In [ ]:
# Amostra dos dados Bronze
print("🔍 Amostra - Bronze Beverage Sales (5 primeiros registros):")
bronze_sales.select(
    "year_month", "region", "brand", "channel_group", 
    "hectoliter", "dollar_sales", "_batch_id"
).show(5, truncate=False)

In [ ]:
# Colunas de Auditoria Bronze
print("📝 Colunas de Auditoria Bronze:")
audit_cols = [c for c in bronze_sales.columns if c.startswith('_')]
bronze_sales.select(audit_cols).show(3, truncate=False)

### 3.2 🥈 Camada SILVER - Dados Limpos e Validados

In [ ]:
# Carregamento dos dados Silver
silver_sales = spark.read.parquet(str(PATHS["silver"] / "beverage_sales"))
silver_channels = spark.read.parquet(str(PATHS["silver"] / "channel_features"))

print("📊 SILVER LAYER - Estatísticas")
print("=" * 50)
print(f"\n🍺 Beverage Sales:")
print(f"   • Total de registros: {silver_sales.count():,}")
print(f"   • Total de colunas: {len(silver_sales.columns)}")
print(f"\n📺 Channel Features:")
print(f"   • Total de registros: {silver_channels.count():,}")
print(f"   • Total de colunas: {len(silver_channels.columns)}")

In [ ]:
# Schema da tabela Silver Sales
print("📋 Schema - Silver Beverage Sales:")
silver_sales.printSchema()

In [ ]:
# Validação de Qualidade - Silver
print("✅ Validação de Qualidade - Silver Layer:")
print("=" * 50)

# Verifica valores nulos em colunas críticas
critical_cols = ["year_month", "region", "brand", "hectoliter", "dollar_sales"]
for col in critical_cols:
    if col in silver_sales.columns:
        null_count = silver_sales.filter(F.col(col).isNull()).count()
        status = "✅" if null_count == 0 else "⚠️"
        print(f"  {status} {col}: {null_count} nulos")

# Verifica valores negativos
print("\n📊 Verificação de valores negativos:")
numeric_cols = ["hectoliter", "dollar_sales"]
for col in numeric_cols:
    if col in silver_sales.columns:
        neg_count = silver_sales.filter(F.col(col) < 0).count()
        status = "✅" if neg_count == 0 else "⚠️"
        print(f"  {status} {col}: {neg_count} negativos")

### 3.3 🥇 Camada GOLD - Dados Enriquecidos com Regras de Negócio

In [ ]:
# Carregamento dos dados Gold
gold_sales = spark.read.parquet(str(PATHS["gold"] / "beverage_sales_enriched"))

print("📊 GOLD LAYER - Estatísticas")
print("=" * 50)
print(f"\n🍺 Beverage Sales Enriched:")
print(f"   • Total de registros: {gold_sales.count():,}")
print(f"   • Total de colunas: {len(gold_sales.columns)}")

In [ ]:
# Schema da tabela Gold
print("📋 Schema - Gold Beverage Sales Enriched:")
gold_sales.printSchema()

In [ ]:
# Amostra dos dados Gold com enriquecimentos
print("🔍 Amostra - Gold com colunas de negócio:")
gold_cols = [c for c in gold_sales.columns if not c.startswith('_')]
gold_sales.select(gold_cols[:12]).show(5, truncate=False)

### 3.4 💎 Camada CONSUMPTION - Modelo Dimensional (Star Schema)

In [ ]:
# Carregamento das Dimensões e Fatos
consumption_path = PATHS["consumption"]

# Dimensões
dim_time = spark.read.parquet(str(consumption_path / "dim_time"))
dim_product = spark.read.parquet(str(consumption_path / "dim_product"))
dim_region = spark.read.parquet(str(consumption_path / "dim_region"))
dim_channel = spark.read.parquet(str(consumption_path / "dim_channel"))

# Fato
fact_sales = spark.read.parquet(str(consumption_path / "fact_sales"))

# Agregações
agg_sales_by_month = spark.read.parquet(str(consumption_path / "agg_sales_by_month"))
agg_sales_by_trade = spark.read.parquet(str(consumption_path / "agg_sales_by_trade_group"))
agg_lowest_brand = spark.read.parquet(str(consumption_path / "agg_lowest_brand_by_region"))

print("📊 CONSUMPTION LAYER - Star Schema")
print("=" * 50)
print("\n📐 DIMENSÕES:")
print(f"   • dim_time: {dim_time.count():,} registros")
print(f"   • dim_product: {dim_product.count():,} registros")
print(f"   • dim_region: {dim_region.count():,} registros")
print(f"   • dim_channel: {dim_channel.count():,} registros")
print(f"\n📈 FATO:")
print(f"   • fact_sales: {fact_sales.count():,} registros")
print(f"\n📊 AGREGAÇÕES:")
print(f"   • agg_sales_by_month: {agg_sales_by_month.count():,} registros")
print(f"   • agg_sales_by_trade_group: {agg_sales_by_trade.count():,} registros")
print(f"   • agg_lowest_brand_by_region: {agg_lowest_brand.count():,} registros")

In [ ]:
# Visualização das Dimensões
print("📐 Dimensão TIME (Tempo):")
dim_time.show(5, truncate=False)

In [ ]:
print("📐 Dimensão PRODUCT (Produto):")
dim_product.show(10, truncate=False)

In [ ]:
print("📐 Dimensão REGION (Região):")
dim_region.show(10, truncate=False)

In [ ]:
print("📐 Dimensão CHANNEL (Canal):")
dim_channel.show(10, truncate=False)

In [ ]:
# Tabela Fato - Schema
print("📈 Tabela FACT_SALES - Schema:")
fact_sales.printSchema()

In [ ]:
# Amostra da Tabela Fato
print("📈 Tabela FACT_SALES - Amostra:")
fact_sales.show(5, truncate=False)

## 📈 4. Business Queries - Respostas aos Requisitos do Case

### 📊 Query 1: Top 3 Trade Groups por Volume de Vendas

In [ ]:
# Query 1: Top 3 Trade Groups
print("🏆 TOP 3 TRADE GROUPS POR VOLUME DE VENDAS (Hectolitros)")
print("=" * 60)

top_trade_groups = agg_sales_by_trade.orderBy(F.desc("total_hectoliter")).limit(3)
top_trade_groups.show(truncate=False)

# Visualização
top_df = top_trade_groups.toPandas()
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(top_df['trade_group'], top_df['total_hectoliter'], color=['gold', 'silver', '#CD7F32'])
ax.set_xlabel('Hectolitros', fontsize=12)
ax.set_title('Top 3 Trade Groups por Volume de Vendas', fontsize=14, fontweight='bold')
ax.invert_yaxis()

# Adiciona valores nas barras
for bar, val in zip(bars, top_df['total_hectoliter']):
    ax.text(bar.get_width() + 1000, bar.get_y() + bar.get_height()/2, 
            f'{val:,.0f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

### 📊 Query 2: Vendas por Mês

In [ ]:
# Query 2: Vendas por Mês
print("📅 VENDAS POR MÊS")
print("=" * 60)

sales_by_month = agg_sales_by_month.orderBy("year_month")
sales_by_month.show(20, truncate=False)

# Visualização
month_df = sales_by_month.toPandas()
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(month_df['year_month'].astype(str), month_df['total_hectoliter'], 
        marker='o', linewidth=2, markersize=8, color='#1f77b4')
ax.fill_between(range(len(month_df)), month_df['total_hectoliter'], alpha=0.3)

ax.set_xlabel('Ano/Mês', fontsize=12)
ax.set_ylabel('Hectolitros', fontsize=12)
ax.set_title('Evolução de Vendas por Mês (Hectolitros)', fontsize=14, fontweight='bold')
plt.xticks(rotation=45)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))

plt.tight_layout()
plt.show()

### 📊 Query 3: Marca com Menor Volume por Região

In [ ]:
# Query 3: Menor Marca por Região
print("📉 MARCA COM MENOR VOLUME DE VENDAS POR REGIÃO")
print("=" * 60)

lowest_brand = agg_lowest_brand.orderBy("region")
lowest_brand.show(truncate=False)

In [ ]:
# Visualização das marcas com menor desempenho
lowest_df = lowest_brand.toPandas()

fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.bar(lowest_df['region'], lowest_df['total_hectoliter'], color='coral', edgecolor='darkred')

# Adiciona labels das marcas nas barras
for bar, brand in zip(bars, lowest_df['brand']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50, 
            brand, ha='center', va='bottom', fontsize=9, rotation=45)

ax.set_xlabel('Região', fontsize=12)
ax.set_ylabel('Hectolitros', fontsize=12)
ax.set_title('Marca com Menor Volume por Região', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()

## 📊 5. Análises Adicionais

### 5.1 Distribuição de Vendas por Região

In [ ]:
# Vendas por Região
sales_by_region = gold_sales.groupBy("region").agg(
    F.sum("hectoliter").alias("total_hectoliter"),
    F.sum("dollar_sales").alias("total_dollar_sales"),
    F.count("*").alias("num_transactions")
).orderBy(F.desc("total_hectoliter"))

print("🌎 VENDAS POR REGIÃO")
print("=" * 60)
sales_by_region.show(truncate=False)

# Visualização - Pie Chart
region_df = sales_by_region.toPandas()

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Pie chart - Volume
axes[0].pie(region_df['total_hectoliter'], labels=region_df['region'], autopct='%1.1f%%',
           colors=plt.cm.Set3.colors[:len(region_df)])
axes[0].set_title('Distribuição de Volume por Região', fontweight='bold')

# Bar chart - Faturamento
axes[1].barh(region_df['region'], region_df['total_dollar_sales'], color='steelblue')
axes[1].set_xlabel('Faturamento ($)')
axes[1].set_title('Faturamento por Região', fontweight='bold')
axes[1].xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))

plt.tight_layout()
plt.show()

### 5.2 Top 10 Marcas por Volume

In [ ]:
# Top 10 Marcas
top_brands = gold_sales.groupBy("brand").agg(
    F.sum("hectoliter").alias("total_hectoliter"),
    F.sum("dollar_sales").alias("total_dollar_sales")
).orderBy(F.desc("total_hectoliter")).limit(10)

print("🏷️ TOP 10 MARCAS POR VOLUME")
print("=" * 60)
top_brands.show(truncate=False)

# Visualização
brands_df = top_brands.toPandas()

fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.barh(brands_df['brand'], brands_df['total_hectoliter'], color='forestgreen')
ax.set_xlabel('Hectolitros', fontsize=12)
ax.set_title('Top 10 Marcas por Volume de Vendas', fontsize=14, fontweight='bold')
ax.invert_yaxis()
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))

plt.tight_layout()
plt.show()

### 5.3 Análise de Sazonalidade

In [ ]:
# Sazonalidade - Vendas por Mês do Ano
seasonality = dim_time.join(fact_sales, "time_key", "inner").groupBy("month").agg(
    F.sum("hectoliter").alias("total_hectoliter"),
    F.avg("hectoliter").alias("avg_hectoliter")
).orderBy("month")

print("📆 SAZONALIDADE - VENDAS POR MÊS DO ANO")
print("=" * 60)
seasonality.show(12, truncate=False)

# Visualização
season_df = seasonality.toPandas()
months = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']

fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(range(1, len(season_df) + 1), season_df['total_hectoliter'], color='royalblue', alpha=0.7)
ax.plot(range(1, len(season_df) + 1), season_df['total_hectoliter'], 'ro-', markersize=8)

ax.set_xlabel('Mês', fontsize=12)
ax.set_ylabel('Hectolitros', fontsize=12)
ax.set_title('Padrão de Sazonalidade de Vendas', fontsize=14, fontweight='bold')
ax.set_xticks(range(1, len(season_df) + 1))
ax.set_xticklabels(months[:len(season_df)])
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))

plt.tight_layout()
plt.show()

### 5.4 Matriz de Vendas: Região x Canal

In [ ]:
# Matriz Região x Canal
region_channel = gold_sales.groupBy("region", "trade_group").agg(
    F.sum("hectoliter").alias("total_hectoliter")
).orderBy("region", "trade_group")

# Pivot para matriz
pivot_df = region_channel.groupBy("region").pivot("trade_group").sum("total_hectoliter")

print("📊 MATRIZ REGIÃO x TRADE GROUP (Hectolitros)")
print("=" * 60)
pivot_df.show(truncate=False)

# Heatmap
matrix_df = pivot_df.toPandas().set_index('region').fillna(0)

fig, ax = plt.subplots(figsize=(14, 8))
sns.heatmap(matrix_df, annot=True, fmt=',.0f', cmap='YlOrRd', ax=ax, 
            linewidths=0.5, cbar_kws={'label': 'Hectolitros'})
ax.set_title('Heatmap: Vendas por Região e Trade Group', fontsize=14, fontweight='bold')
ax.set_xlabel('Trade Group', fontsize=12)
ax.set_ylabel('Região', fontsize=12)
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()

## 🔍 6. Controle de Qualidade e Métricas do Pipeline

In [ ]:
# Carrega dados de controle
try:
    process_control = spark.read.parquet(str(PATHS["control"] / "process_control"))
    quarantine = spark.read.parquet(str(PATHS["control"] / "quarantine"))
    
    print("📋 CONTROLE DE PROCESSAMENTO")
    print("=" * 60)
    process_control.show(truncate=False)
    
    print("\n⚠️ REGISTROS EM QUARENTENA (Rejeitados)")
    print("=" * 60)
    quarantine_count = quarantine.count()
    print(f"Total de registros em quarentena: {quarantine_count}")
    if quarantine_count > 0:
        quarantine.show(5, truncate=False)
except Exception as e:
    print(f"ℹ️ Dados de controle não disponíveis: {e}")

In [ ]:
# Estatísticas Gerais do Pipeline
print("📊 RESUMO ESTATÍSTICO DO PIPELINE")
print("=" * 60)

stats = {
    "Camada": ["Bronze", "Silver", "Gold", "Consumption (Fact)"],
    "Tabela": ["beverage_sales", "beverage_sales", "beverage_sales_enriched", "fact_sales"],
    "Registros": [
        bronze_sales.count(),
        silver_sales.count(),
        gold_sales.count(),
        fact_sales.count()
    ]
}

stats_df = pd.DataFrame(stats)
print(stats_df.to_string(index=False))

# Verifica integridade
print("\n✅ VERIFICAÇÃO DE INTEGRIDADE:")
if bronze_sales.count() >= silver_sales.count() >= gold_sales.count():
    print("  ✅ Registros mantidos/filtrados corretamente entre camadas")
else:
    print("  ⚠️ Verifique a integridade entre camadas")

## 📋 7. Sumário Executivo

In [ ]:
# Cálculos para o sumário
total_hectoliter = gold_sales.agg(F.sum("hectoliter")).collect()[0][0]
total_revenue = gold_sales.agg(F.sum("dollar_sales")).collect()[0][0]
num_brands = gold_sales.select("brand").distinct().count()
num_regions = gold_sales.select("region").distinct().count()
num_channels = gold_sales.select("trade_group").distinct().count()
date_range = gold_sales.agg(F.min("year_month"), F.max("year_month")).collect()[0]

print("="*70)
print("                    📊 SUMÁRIO EXECUTIVO - ABInBev Case")
print("="*70)
print(f"""
📅 PERÍODO DE ANÁLISE
   • De: {date_range[0]} até {date_range[1]}

📈 MÉTRICAS PRINCIPAIS
   • Volume Total: {total_hectoliter:,.0f} hectolitros
   • Faturamento Total: ${total_revenue:,.2f}
   • Ticket Médio: ${total_revenue/total_hectoliter:.2f}/hectolitro

🏷️ DIMENSÕES
   • Marcas: {num_brands} únicas
   • Regiões: {num_regions} únicas
   • Trade Groups: {num_channels} únicos

🏗️ ARQUITETURA
   • Camadas: Landing → Bronze → Silver → Gold → Consumption
   • Formato: Parquet (compatível com Delta Lake)
   • Modelo: Star Schema (Dimensões + Fatos)

✅ STATUS DO PIPELINE
   • Bronze: {bronze_sales.count():,} registros ingeridos
   • Silver: {silver_sales.count():,} registros validados
   • Gold: {gold_sales.count():,} registros enriquecidos
   • Fact: {fact_sales.count():,} transações
""")
print("="*70)

## 🔚 8. Encerramento

In [ ]:
# Encerra a sessão Spark
print("🔚 Encerrando sessão Spark...")
spark.stop()
print("✅ Sessão encerrada com sucesso!")
print(f"\n📅 Apresentação finalizada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

---

## 📚 Referências

- **Arquitetura:** [ARCHITECTURE.md](../docs/ARCHITECTURE.md)
- **Dicionário de Dados:** [DATA_DICTIONARY.md](../docs/DATA_DICTIONARY.md)
- **Código Fonte:** [src/](../src/)
- **Testes:** [tests/](../tests/)

---

**🍺 ABInBev Data Engineering Case**  
*Desenvolvido por Michael Santos*